In [ ]:
using RCall
using BenchmarkTools
using CSV
using DataFrames
using GLM
using PenalizedGLMM
using SnpArrays

In [ ]:
pkgversion(PenalizedGLMM)

In [ ]:
pkgversion(BenchmarkTools)

In [ ]:
Threads.nthreads()

In [ ]:
### data filepaths ###
const datadir = "data/"
const covfile = datadir * "penncath.csv" # includes binary phenotype data
const plinkfile = datadir * "qc_penncath"
const grmfile = datadir * "PennCathGRM.txt.gz" # genetic relatedness matrix

In [ ]:
### read in covariate file ###
covdf = CSV.read(covfile, DataFrame)
#fam = CSV.read(famfile, DataFrame, delim = " ", header = false)
#rename!(fam,:Column1 => :FamID)
#covdf = rightjoin(covdf, fam, on = :FamID)

In [ ]:
# force CAD to be a float (for normal approx)
covdf[:,:CADfloat] = convert(Array{Float64}, covdf[:,:CAD])
CSV.write("penncathfloat.csv", covdf)
const floatcovfile = "penncathfloat.csv"

In [ ]:
# Run the benchmark 10 times
# normnull_times = []
# for i in 1:10
#     push!(normnull_times, @belapsed pglmm_null(@formula(CADfloat ~ age + sex), covfile = floatcovfile, grmfile = grmfile, 
#    family = Normal(), link = IdentityLink()))
# end

# Calculate the median and range
# median_time = median(normnull_times)
# min_time = minimum(normnull_times)
# max_time = maximum(normnull_times)

# println("Median time: $median_time seconds")
# println("Minimum: $min_time seconds")
# println("Maximum: $max_time seconds")

In [ ]:
nullmodel_normal = pglmm_null(@formula(CADfloat ~ sex + age), covfile = floatcovfile, grmfile = grmfile, 
   family = Normal(), link = IdentityLink())

In [ ]:
@elapsed pglmm_null(@formula(CADfloat ~ sex + age), covfile = floatcovfile, grmfile = grmfile, 
   family = Normal(), link = IdentityLink())

In [ ]:
print(nullmodel_normal.converged)

In [ ]:
# Normal model

# Run the benchmark 10 times
# normfit_times = []
# for i in 1:10
#     push!(normfit_times, @belapsed pglmm(nullmodel_normal, plinkfile))
# end

# Calculate the median and range
# median_time = median(normfit_times)
# min_time = minimum(normfit_times)
# max_time = maximum(normfit_times)

# println("Median time: $median_time seconds")
# println("Minimum: $min_time seconds")
# println("Maximum: $max_time seconds")

In [ ]:
modelfit_normal = pglmm(nullmodel_normal, plinkfile)

In [ ]:
@elapsed pglmm(nullmodel_normal, plinkfile)